# Backtest the strategies

Use an LLM to go through and predict the buy/ sell/ hold recommendation for the company for the given date. Steps needed:

1. Load the LLM - use DeepSeek R1 Qwen model at 7B parameters first and try the quantised models next
2. Step through each data and each financial statement to get a result
3. Log the results in a file and save to S3 (will need a logging file to save to S3 and resume in case of kernel crash)
4. Need a backtesting framework to apply the results


## Load libraries needed

In [3]:
import json
import boto3
from s3fs import S3FileSystem
import os

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import torch

import pandas as pd

from IPython.display import Markdown, display

from helper import get_s3_folder
import s3_model
from s3_model import S3ModelHelper

## Load the LLM

Models to test:
- Qwen (Qwen/Qwen2.5-7B-Instruct)
- Llama (meta-llama/Llama-3.2-7B-Instruct)
- DeepSeek (deepseek-ai/DeepSeek-R1-Distill-Qwen-14B)

In [2]:
# Log into Huggingface

with open('pass.txt') as p:
    hf_login = p.read()
    
hf_login = hf_login[hf_login.find('=')+1:hf_login.find('\n')]
login(hf_login, add_to_git_credential=False)

In [10]:
c = s3_model.S3ModelHelper(s3_sub_folder='tmp/fs')
c.clear_folder('deepseek')

In [11]:
# Flag to download from Huggingface again or use stored model
USE_HF = False

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
model_id_s3 = 'deepseek'


if USE_HF:
   
    pipeline = transformers.pipeline(
        "text-generation",
        model=model_id,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="auto",
    )

    model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.bfloat16 )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
else:
    qwen = s3_model.S3ModelHelper(s3_sub_folder='tmp/fs')
    model = qwen.load_model(model_id_s3)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="auto",
    )
    qwen.clear_folder(model_id_s3)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Device set to use cuda:0
